In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from multiprocessing import Pool
import multiprocessing
import multiprocess as mp
# import worker

import sqlite3

import json, requests, os, urllib

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/switrs.sqlite")
q = """
SELECT collision_date, longitude, latitude FROM collisions
WHERE collision_date IS NOT NULL
AND longitude IS NOT NULL
AND latitude IS NOT NULL    
AND collision_date <= '2017-12-31'  -- 2018 is incomplete
AND collision_date >= '2010-01-01'
"""
df = pd.read_sql_query(q, con)
df.latitude = round(df.latitude)
df.longitude = round(df.longitude)
df.drop_duplicates(inplace=True)
# df.collision_date = df.collision_date.str.replace('-','')
# tuples = [tuple(x) for x in df.to_numpy()]

# tupless = pd.read_csv('data/tuples.csv')
# tuples = [tuple(x) for x in tupless.to_numpy()]

In [4]:
df.latitude.min(), df.latitude.max()

(32.0, 42.0)

In [5]:
# So therefore we turn to the regional data as offered by NASA and specified in their API documentation
# This essentially creates a CSV file for every 4.5 x 4.5 block of long/lat combinations, thus in our case creating
# 3 x 3 = 9 seperate files. These files are merged in another Notebook.

# First, create the 4.5 x 4.5 blocks we want to use
longi = [(df.longitude.min(), df.longitude.min() + 4.5), (df.longitude.min() + 4.5, df.longitude.min() + 4.5*2), (df.longitude.min() + 4.5*2, df.longitude.min() + 4.5*3)]
lati = [(df.latitude.min(), df.latitude.min() + 4.5), (df.latitude.min() + 4.5, df.latitude.min() + 4.5*2), (df.latitude.min() + 4.5*2, df.latitude.min() + 4.5*3)]

# Then, request the data and write to CSV
output_folder = os.getcwd() + '/data/large'
output = "CSV" # JSON, CSV, ASCII, ICASA, NETCDF
base_url = r"https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?&request=execute&tempAverage=DAILY&identifier=Regional&parameters=T2M&userCommunity=AG&startDate=20100101&endDate=20171231&bbox={lat0},{long0},{lat1},{long1}&outputList=CSV&user=DOCUMENTATION"
for long in longi:
    for lat in lati:
        api_request_url = base_url.format(long0=long[0], long1=long[1], lat0=lat[0], lat1=lat[1], output=output.upper())

        # Python Memory Object
        json_response = json.loads(requests.get(api_request_url).content.decode('utf-8'))

        # Selects the file URL from the JSON response
        csv_request_url = json_response['outputs'][output.lower()]

        # Download File to Folder
        output_file_location = os.path.join(output_folder, os.path.basename(csv_request_url))
        urllib.request.urlretrieve(csv_request_url, output_file_location)